## Реализация алгоритма градиентного бустинга

Реализуем средствами Python алгоритм градиентного бустинга для деревьев решений.

Реализация деревьев решений была дважды продемонстрирована в предыдущих уроках, в этом не будем ее повторять и возьмем готовую реализацию дерева решений для регрессии из библиотеки `sklearn`.

In [1]:
from sklearn.tree import DecisionTreeRegressor

from sklearn import model_selection
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

Используем один из "игрушечных" датасетов из той же библиотеки.

In [2]:
from sklearn.datasets import load_diabetes

In [3]:
X, y = load_diabetes(return_X_y=True)

Разделим выборку на обучающую и тестовую в соотношении 75/25.

In [4]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

Напишем функцию, реализующую предсказание в градиентном бустинге.

In [5]:
def gb_predict(X, trees_list, coef_list, eta):
    # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
    # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
    return np.array([sum([eta *coef * alg.predict([x])[0] for alg, coef in zip(trees_list, coef_list)]) for x in X])

В качестве функционала ошибки будем использовать среднеквадратичную ошибку. Реализуем соответствующую функцию.

In [6]:
def mean_squared_error(y_real, prediction):
    return (sum((y_real - prediction)**2)) / len(y_real)

Используем $L_{2}$ loss $L(y, z) = (y-z)^{2},$ ее производная по $z$ примет вид $L'(y, z) = 2(z-y)$. Реализуем ее также в виде функции (коэффициент 2 можно отбросить).

In [7]:
def bias(y, z):
    return (y - z)

Реализуем функцию обучения градиентного бустинга.

In [8]:
def gb_fit(n_trees, max_depth, X_train, X_test, y_train, y_test, coefs, eta):
    
    # Деревья будем записывать в список
    trees = []
    
    # Будем записывать ошибки на обучающей и тестовой выборке на каждой итерации в список
    
    for i in range(n_trees):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)

        # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
        # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
        if len(trees) == 0:
            # обучаем первое дерево на обучающей выборке
            tree.fit(X_train, y_train)
                        
        else:
            # Получим ответы на текущей композиции
            target = gb_predict(X_train, trees, coefs, eta)
            
            # алгоритмы начиная со второго обучаем на сдвиг
            tree.fit(X_train, bias(y_train, target))
            
            
        trees.append(tree)
        
    return trees

Теперь обучим несколько моделей с разными параметрами и исследуем их поведение.

In [9]:
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

eta = 0.1
n_trees_list = [1, 5, 10, 12, 25, 50, 100]
max_depth_list = [1, 2, 3, 4, 5, 10, 50]

def serial_fit_zip(n_trees_list, max_depth_list):
    
    list_error_train = []
    list_error_test = []
        
    for i in range(len(n_trees_list)):
        coefs = [1] * n_trees_list[i]
        
        for j in range(len(max_depth_list)):
        
            trees = gb_fit(n_trees_list[i], max_depth_list[j], X_train, X_test, y_train, y_test, coefs, eta)

            train_prediction = gb_predict(X_train, trees, coefs, eta)
            list_error_train.append([n_trees_list[i], max_depth_list[j], mean_squared_error(y_train, train_prediction)])

            test_prediction = gb_predict(X_test, trees, coefs, eta)
            list_error_test.append([n_trees_list[i], max_depth_list[j], mean_squared_error(y_test, test_prediction)])
    
    return list_error_train, list_error_test

In [10]:
list_error_train_z, list_error_test_z = serial_fit_zip(n_trees_list, max_depth_list)

In [11]:
list_error_train_z = np.array(list_error_train_z)
list_error_test_z = np.array(list_error_test_z)

In [12]:
import plotly
import plotly.graph_objs as go

fig1 = go.Scatter3d(x=list_error_train_z.transpose()[0],
                    y=list_error_train_z.transpose()[1],
                    z=list_error_train_z.transpose()[2],
                    marker=dict(opacity=0.9,
                                reversescale=True,
                                colorscale='Blues',
                                size=5),
                    line=dict (width=0.02),
                    mode='markers')


#Make Plot.ly Layout
mylayout = go.Layout(scene=dict(xaxis=dict( title="n_trees"),
                                yaxis=dict( title="max_depth"),
                                zaxis=dict(title="train_error")),)

#Plot and save html
plotly.offline.plot({"data": [fig1],
                     "layout": mylayout},
                     auto_open=True,
                     filename=("3DPlot_train.html"))

'3DPlot_train.html'

<img src="train_1.png" style="width: 500px;">
<a href="3DPlot_train.html">Интерактивный просмотр</a>

In [25]:
fig1 = go.Scatter3d(x=list_error_test_z.transpose()[0],
                    y=list_error_test_z.transpose()[1],
                    z=list_error_test_z.transpose()[2],
                    marker=dict(opacity=0.9,
                                reversescale=True,
                                colorscale='Blues',
                                size=5),
                    line=dict (width=0.02),
                    mode='markers')


#Make Plot.ly Layout
mylayout = go.Layout(scene=dict(xaxis=dict( title="n_trees"),
                                yaxis=dict( title="max_depth"),
                                zaxis=dict(title="train_error")),)

#Plot and save html
plotly.offline.plot({"data": [fig1],
                     "layout": mylayout},
                     auto_open=True,
                     filename=("3DPlot_test.html"))

'3DPlot_test.html'

<img src="test_1.png" style="width: 500px;">
<a href="3DPlot_test.html">Интерактивный просмотр</a>

*Заментно, что алгоритм значительно переобучился - наилучший результат на тестовой выборки достигнут с глубиной дерева 3 и количством деревьев 50*